# Imports

In [1]:
import pm4py
import pandas as pd
from TreeToTptp import TreeToTptp
from ProverWrapper import use_provers
from create_thesis_with_2_spec import create_thesis_with_2_spec, LogicFunctions
import os

In [2]:
def print_provers(provers):
    print(provers[0])
    print(f'Vampire: {provers[1]}')
    print(f'Eprover: {provers[2]}')


def provers_to_csv(provers):
    print(','.join(provers))

In [3]:
def create_problem_folder(folder_name, expr, spec, tptp, vampire, eprover):
    os.makedirs(folder_name, exist_ok=True)
    with open(f'{folder_name}/expression.txt', 'w') as f:
        f.write(expr)
    with open(f'{folder_name}/specification.txt', 'w') as f:
        f.write(spec)
    with open(f'{folder_name}/tptp.p', 'w') as f:
        f.write(tptp)
    with open(f'{folder_name}/vampire.txt', 'w') as f:
        f.write(vampire)
    with open(f'{folder_name}/eprover.txt', 'w') as f:
        f.write(eprover)

# Julia's problems

In [4]:
folder = "test"

log = pm4py.read_xes("Data/running-example.xes")

process_tree = pm4py.discover_process_tree_inductive(
    log, 0.0, activity_key='concept:name', case_id_key='case:concept:name', timestamp_key='time:timestamp')
expr, spec, tptp = TreeToTptp().tree_to_tptp(process_tree)
vampire, eprover = use_provers(f'{folder}/tptp.p')
create_problem_folder(folder, expr, spec, tptp, vampire, eprover)

/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 6/6 [00:00<00:00, 2335.14it/s]

Seq3(1]register_request, Loop(2]l_s0, Seq2(3]And2(4]a2_s0, check_ticket, Xor2(5]x2_s0, examine_casually, examine_thoroughly, x2_e0[5), a2_e0[4), decide[3), reinitiate_request[2), Xor2(2]x2_s1, pay_compensation, reject_request, x2_e1[2)[1)
ini: register_request
fin: x2_e1



/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/conversion/log/variants/to_data_frame.py:73: UserWarning: no explicit representation of timezones available for np.datetime64
  ev[attr] = np.datetime64(ev[attr])
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
/home/jakub/miniconda3/envs/sat/lib/python3.13/site-packages/pm4py/objects/log/util/d